## Getting ready...

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

2022-01-22 13:46:44.758661: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
import tensorflow.keras as Keras

## How to do it

In [6]:
def define_feature_columns_layer(data_df, categorical_cols, numeic_cols):
    feature_columns = []
    feature_layer_inputs = {}
    
    for feature_name in numeric_cols:
        feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
        feature_layer_inputs[feature_name] = tf.keras.Input(shape=(1,), name=feature_name)
        
    for feature_name in categorical_cols:
        vocabulary = data_df[feature_name].unique()
        cat = tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)
        cat_one_hot = tf.feature_column.indicator_column(cat)
        feature_columns.append(cat_one_hot)
        feature_layer_inputs[feature_name] = tf.keras.Input(shape=(1,), name=feature_name, dtype=tf.int32)
        
    return feature_columns, feature_layer_inputs

In [7]:
def create_interactions(interactions_list, buckets=5):
    feature_columns = []
    
    for (a, b) in interactions_list:
        crossed_feature = tf.feature_column.crossed_column([a, b], hash_bucket_size=buckets)
        crossed_feature_one_hot = tf.feature_column.indicator_column(crossed_feature)
        feature_columns.append(crossed_feature_one_hot)
        
    return feature_columns

In [21]:
def create_linreg(feature_columns, feature_layer_inputs, optimizer):
    
    feature_layer = Keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = Keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = Keras.layers.Dense(1, kernel_initializer='normal', activation='linear')(norm)
    
    model = Keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [23]:
housing_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
path = tf.keras.utils.get_file(housing_url.split('/')[-1], housing_url)

data = pd.read_table(path, delim_whitespace=True, header=None, names=columns)

In [24]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns, feature_layer_inputs = define_feature_columns_layer(data, categorical_cols, numeric_cols)
interaction_columns = create_interactions([['RM', 'LSTAT']])

feature_columns += interaction_columns

optimizer = Keras.optimizers.Ftrl(learning_rate=0.02)
model = create_linreg(feature_columns, feature_layer_inputs, optimizer)

In [25]:
import tempfile

def canned_keras(model):
    model_dir = tempfile.mkdtemp()
    keras_estimator = tf.keras.estimator.model_to_estimator(keras_model=model, model_dir=model_dir)
    return keras_estimator
estimator = canned_keras(model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/wil/tensorflow/venv/lib/python3.8/site-packages/tensorflow/python/keras/backend.py:435: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
2022-01-22 14:10:01.492469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 14:10:01.492705: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-22 14:10:01.492751: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful 

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfm8nazjz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
np.random.seed(1)
train = data.sample(frac=0.8).copy()
y_train = train['MEDV']
train.drop('MEDV', axis=1, inplace=True)

test = data.loc[~data.index.isin(train.index)].copy()
y_test = test['MEDV']
test.drop('MEDV', axis=1, inplace=True)

In [27]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=256):
    
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    
    return input_function

In [28]:
train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmpfm8nazjz/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: /tmp/tmpfm8nazjz/keras/keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 4 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2022-01-22 14:12:04.992147: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 14:12:04.992398: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-22 14:12:04.992444: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 14:12:04.992672: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 14:12:04.992871: I tensorflow/core/common

INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpfm8nazjz/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...


2022-01-22 14:12:05.310810: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


INFO:tensorflow:loss = 587.195, step = 0


2022-01-22 14:12:05.807000: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


INFO:tensorflow:global_step/sec: 437.93
INFO:tensorflow:loss = 318.49603, step = 100 (0.229 sec)
INFO:tensorflow:global_step/sec: 546.634
INFO:tensorflow:loss = 139.72025, step = 200 (0.183 sec)
INFO:tensorflow:global_step/sec: 555.399
INFO:tensorflow:loss = 59.24311, step = 300 (0.180 sec)
INFO:tensorflow:global_step/sec: 539.006
INFO:tensorflow:loss = 45.963875, step = 400 (0.186 sec)
INFO:tensorflow:global_step/sec: 560.652
INFO:tensorflow:loss = 37.375244, step = 500 (0.178 sec)
INFO:tensorflow:global_step/sec: 561.599
INFO:tensorflow:loss = 32.92877, step = 600 (0.178 sec)
INFO:tensorflow:global_step/sec: 555.694
INFO:tensorflow:loss = 29.270836, step = 700 (0.180 sec)
INFO:tensorflow:global_step/sec: 560.28
INFO:tensorflow:loss = 25.946384, step = 800 (0.179 sec)
INFO:tensorflow:global_step/sec: 558.796
INFO:tensorflow:loss = 31.557884, step = 900 (0.179 sec)
INFO:tensorflow:global_step/sec: 535.274
INFO:tensorflow:loss = 24.583134, step = 1000 (0.187 sec)
INFO:tensorflow:global_

/home/wil/tensorflow/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2022-01-22 14:12:11.071849: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 14:12:11.072091: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-22 14:12:11.072136: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1

In [30]:
weights = estimator.get_variable_value('layer_with_weights-1/kernel/.ATTRIBUTES/VARIABLE_VALUE')
print(weights)

[[-0.8442589 ]
 [-0.9196228 ]
 [-0.8522259 ]
 [ 0.87031704]
 [ 0.83735174]
 [ 1.1116372 ]
 [-0.80827796]
 [-1.2430266 ]
 [ 0.87397623]
 [-0.8844386 ]
 [-0.91382885]
 [-0.9379465 ]
 [-0.89458334]
 [-0.8403668 ]
 [-0.99148303]
 [-0.91806936]
 [ 0.9299024 ]
 [ 0.93877435]
 [ 0.8835228 ]
 [ 0.92570865]
 [-0.83936316]
 [ 0.8989944 ]
 [-0.8777855 ]
 [ 0.90241015]
 [ 1.2515244 ]
 [-0.7917902 ]
 [-0.8693719 ]]


In [32]:
def extract_labels(feature_columns):
    labels=list()
    for col in feature_columns:
        col_config = col.get_config()
        if 'key' in col_config:
            labels.append(col_config['key'])
        elif 'categorical_column' in col_config:
            if col_config['categorical_column']['class_name'] == 'VocabularyListCategoricalColumn':
                key = col_config['categorical_column']['config']['key'] 
                for item in col_config['categorical_column']['config']['vocabulary_list']: 
                    labels.append(key+'_val='+str(item))
            elif col_config['categorical_column']['class_name'] == 'CrossedColumn':
                keys = col_config['categorical_column']['config']['keys']
                for bucket in range(col_config['categorical_column']['config']['hash_bucket_size']):
                    labels.append('x'.join(keys)+'_bkt_'+str(bucket))
    return labels

In [33]:
labels = extract_labels(feature_columns)

for label, weight in zip(labels, weights):
    print(f"{label:15s} : {weight[0]:+.2f}")

CRIM            : -0.84
ZN              : -0.92
INDUS           : -0.85
NOX             : +0.87
RM              : +0.84
AGE             : +1.11
DIS             : -0.81
TAX             : -1.24
PTRATIO         : +0.87
B               : -0.88
LSTAT           : -0.91
CHAS_val=0      : -0.94
CHAS_val=1      : -0.89
RAD_val=1       : -0.84
RAD_val=2       : -0.99
RAD_val=3       : -0.92
RAD_val=5       : +0.93
RAD_val=4       : +0.94
RAD_val=8       : +0.88
RAD_val=6       : +0.93
RAD_val=7       : -0.84
RAD_val=24      : +0.90
RMxLSTAT_bkt_0  : -0.88
RMxLSTAT_bkt_1  : +0.90
RMxLSTAT_bkt_2  : +1.25
RMxLSTAT_bkt_3  : -0.79
RMxLSTAT_bkt_4  : -0.87
